In [2]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
# 사용할 LLM 모델 설정
llm = Ollama(model="gemma2")

/tmp/ipykernel_452767/3434335749.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2")


In [5]:
llm.invoke("뉴스기사 도메인으로는 어떤 게 있어?")

'뉴스 기사 도메인은 매우 광범위하며 다양한 분야를 포괄합니다. \n\n**주요 뉴스 기사 도메인들은 다음과 같습니다:**\n\n* **정치:** 정부, 정당, 선거, 국제 관계 등 정치적 이슈에 대한 보도\n* **경제:** 금융 시장, 경제 성장, 실업률, 물가 변동 등 경제 활동 관련 뉴스\n* **사회:** 범죄, 사회 문제, 교육, 문화, 건강, 환경 등 사회적인 현상과 이슈에 대한 보도\n* **생활:** 날씨 예보, 요리법, 여행 정보, 엔터테인먼트 소식 등 일상 생활에 관련된 뉴스\n* **IT/기술:** 인공지능, 빅데이터, 모바일 기술, 게임 등 IT 및 기술 분야의 발전과 트렌드\n* **스포츠:**  다양한 스포츠 경기 결과, 선수 정보, 용단 전략 등 스포츠 관련 뉴스\n* **국제:** 해외 정세, 국제 사건, 외교 관계 등 글로벌 이슈에 대한 보도\n\n**추가적으로, 특정 지역이나 주제를 다루는 뉴스 기사 도메인도 존재합니다.** 예를 들어:\n\n* **지역 뉴스:**  특정 도시 또는 지역의 현안 문제 및 이벤트에 집중하는 뉴스\n* **전문 분야 뉴스:** 의학, 법률, 금융 등 특정 분야 전문 지식을 제공하는 뉴스\n* **기업 뉴스:** 기업의 경영상황, 제품 출시, 시장 동향 등 기업 관련 정보를 보도하는 뉴스\n\n\n어떤 분야에 관심이 있는지에 따라 자신에게 맞는 뉴스 기사 도메인을 선택하여 업데이트 받을 수 있습니다.'

In [16]:
llm.invoke("뉴스기사 도메인의 종류를 간단하게 항목만 추려서 리스트에 담아줘.")

'## 뉴스 기사 도메인 (항목)\n\n* 정치\n* 경제\n* 사회\n* 문화\n* 예술\n* 과학\n* 기술\n* 범죄\n* 스포츠\n* 국제\n* 건강\n* 교육\n* 여가\n\n\n'

In [18]:
llm.invoke("뉴스기사 도메인의 종류를 간단하게 항목만 10개정도 추려서 리스트에 담아줘.")

'## 뉴스 기사 도메인 (항목)\n\n1. 정치\n2. 경제\n3. 사회\n4. 문화\n5. 과학·기술\n6. 스포츠\n7. 범죄 및 법정\n8. 국제\n9. 여행\n10. 건강  \n\n\n'

In [19]:
llm.invoke("뉴스기사 도메인의 종류를 간단하게 항목만 8개정도 추려서 리스트에 담아줘.")

'## 뉴스기사 도메인 (8가지)\n\n1. 정치\n2. 경제\n3. 사회\n4. 문화\n5. 스포츠\n6. 국제\n7. IT/과학\n8. 건강 \n\n\n'

In [20]:
llm.invoke("뉴스기사 도메인의 종류를 간단하게 항목만 7개정도 추려서 리스트에 담아줘.")

'## 뉴스 기사 도메인 (7가지)\n\n1. 정치\n2. 경제\n3. 사회\n4. 문화\n5. 국제\n6. 스포츠\n7. 과학/기술 \n\n\n'

In [21]:
domains = ['정치', '경제', '사회', '문화', '국제', '스포츠', '과학/기술']

In [24]:
import pandas as pd
from typing import Dict, List
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser


# JSONOutputParser 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "당신은 주어진 도메인에 대한 기사 제목을 생성하는 어시스턴트입니다. 각 도메인마다 한 문장 내외의 적절한 뉴스 기사제목을 생성해야 합니다."),
    ("user", 
     "#Format: {format_instructions}\n\n#Requirements: {req}\n\n#Domain: {domain}")
])

# 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인 생성
chain = prompt | llm | parser

def generate_domain_titles(domains: List[str]) -> Dict[str, List[str]]:
    """
    여러 도메인에 대한 뉴스 제목을 생성하고 DataFrame으로 반환하는 함수
    
    Args:
        domains (List[str]): 뉴스 도메인 리스트
    
    Returns:
        pd.DataFrame: target(도메인)과 text(제목) 컬럼을 가진 DataFrame
    """
    data = []
    req = "입력된 도메인은 `domain`에, 출력된 뉴스기사 제목은 `titles` 리스트에 100개씩 담아주세요."
    
    try:
        for domain in domains:
            print(f"\n{domain} 도메인의 뉴스 제목 생성 중...")
            response = chain.invoke({"req": req, "domain": domain})
            
            # 각 제목을 개별 행으로 추가
            for title in response.get('titles', []):
                data.append({
                    'target': domain,
                    'text': title
                })
            
    except Exception as e:
        print(f"오류 발생: {str(e)}")
    
    return pd.DataFrame(data)

In [28]:
# 뉴스 제목 생성
df1 = generate_domain_titles(domains)


정치 도메인의 뉴스 제목 생성 중...

경제 도메인의 뉴스 제목 생성 중...

사회 도메인의 뉴스 제목 생성 중...

문화 도메인의 뉴스 제목 생성 중...

국제 도메인의 뉴스 제목 생성 중...

스포츠 도메인의 뉴스 제목 생성 중...

과학/기술 도메인의 뉴스 제목 생성 중...


In [50]:
# CSV 파일로 저장
csv_filename = 'news_titles_domains_temp.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"\nCSV 파일이 {csv_filename}로 저장되었습니다.")


CSV 파일이 news_titles_domains_temp.csv로 저장되었습니다.


In [ ]:
domain_all = pd.concat([df_domain, df], ignore_index=True)
domain_all

,target,text
0,정치,새로운 정책 발표로 국민 반응은?
1,정치,"대선 후보, 경제 공약 확대 전략 추진"
2,정치,"국회 긴급한 회의 소집, 중요 법안 논의 예고"
3,정치,"야당, 과행정 비판 강화 '총력' mobilze"
4,정치,지방 선거 결과 분석: 여당 지지율 변동 주목
...,...,...
2013,스포츠,"AI 기술 활용, 스포츠 분석 시스템 고도화"
2014,스포츠,"스포츠 기업의 새로운 전략, '팬들과 소통하는 방식'"
2015,스포츠,"사회적 책임, 스포츠 행사에서 중요성 부각"
2016,스포츠,"지속 가능한 스포츠 발전, '환경 보호'와 함께"


In [59]:
domain_all.to_csv('new_title_domains.csv')

## 도메인을 알아서 추리고, 그에 알맞는 뉴스기사 제목 생성

In [ ]:
from langchain.prompts import PromptTemplate
from typing import List


# 프롬프트 템플릿 설정 - 좀 더 구체적인 지시사항 추가
prompt = PromptTemplate(
    input_variables=[],
    template="""다음 주어지는 순서에 따라, 도메인별로 뉴스 기사 제목을 5개 생성해주세요:
    1. 뉴스기사 도메인 종류를 간단하게 추려주세요.
    2. 추려진 도메인에 맞는 뉴스기사 헤드라인 5개씩 생성해주세요.
    - 각 제목은 실제 뉴스처럼 구체적이고 현실적이어야 합니다.
    - 제목은 한 줄에 하나씩, 번호를 붙여서 작성합니다.
    - 클릭베이트나 과장된 표현 사용하지 않습니다.
    - 출력은 ['도메인 종류': '뉴스 기사 제목'] 형식입니다.
    """
)


def generate_news_titles() -> str:
    """
    특정 도메인에 대한 뉴스 제목을 생성하는 함수
    
    Args:
        domain (str): 뉴스 도메인 (예: 정치, 경제, 사회 등)
    
    Returns:
        str: 생성된 뉴스 제목들
    """
    try:
        # 프롬프트 생성
        formatted_prompt = prompt.format()
        
        # LLM으로 제목 생성
        response = llm.invoke(formatted_prompt)
        
        return response  # 직접 response 반환
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return ""

titles = generate_news_titles()

In [ ]:
titles

'## 뉴스 기사 헤드라인 생성\n\n**1. 경제:**\n\n1. 금리 인상 가능성에 대한 우려, 전망기업 주가 하락세 확산\n2. LG전자, 새로운 롤러형 OLED TV 출시… 시장 선점 노력 가속화\n3. 한국수출 동향 개선, 미국의 수요 회복 이끄는 주역으로 부상\n4. 삼성SDS, 클라우드 인프라 확대 및 AI 전문 서비스 제공 강화\n5. 글로벌 투자 유치 활성화, 국내 벤처기업 성장 동력\n\n**2. 정치:**\n\n1. 대선 이후 최초 총정례회, 국민적 합의와 국제협력 주시\n2. 반추리 정책 논란 불거짐, 의원들 간 열띤 토론 및 조율 추진\n3. 법률 개혁 촉구 시위 진행, 사회적 불평등 해소를 위한 대안 제시\n4. 북한의 최근 군사 행동에 대한 국제 사회 경각심 고조, 긴장 완화 노력 강화\n5. 외교 관계 개선을 위한 다자간 회담 계획, 한미일 공동 방위 의지 확인\n\n**3. 사회:**\n\n1. 온라인 성추행 사건 증가에 대한 대응책 논의 활발, 법 개정 필요성 제기\n2. 고령화 사회 심각성 인식, 노인복지 정책 강화 위한 논의 시작\n3. 지역 소통 및 공동체 의식 회복을 위한 시민참여 프로그램 확대 추진\n4. 환경오염 문제에 대한 국민적 관심 증가, 지속 가능한 발전 방안 모색\n5. 교육열 고조, 청소년들의 학업 스트레스 심화… 정신 건강 지원 필요성 강조\n\n**4. 문화:**\n\n1. 한국 영화, 해외 시장 진출 확대… 글로벌 팬덤 확보 노력\n2. 국내 음악 예술가, 세계적 뮤직 페스티벌 참여 통해 국제적인 위상 제고\n3. 인공지능 활용한 새로운 창작 콘텐츠 개발, 문화 산업 발전 동력으로 부상\n4. 전통문화 체험 프로그램 확대, 국내 관광산업 활성화에 기여\n5. 독립영화 지원 및 공개 시스템 개선, 다양성 있는 영화 제작 환경 조성\n\n**5. 과학:**\n\n1. 한국 연구팀, 암 치료 효과 발휘하는 신약 개발 성공… 임상시험 진행 예정\n2. 인공지능 기술 활용한 의료진 보조 시스템 개발 추진, 의료

In [20]:
import pandas as pd

# 뉴스 기사 데이터 (도메인별 제목을 리스트로 구성)
data = {
    "경제": [
        "금리 인상 가능성에 대한 우려, 전망기업 주가 하락세 확산",
        "LG전자, 새로운 롤러형 OLED TV 출시… 시장 선점 노력 가속화",
        "한국수출 동향 개선, 미국의 수요 회복 이끄는 주역으로 부상",
        "삼성SDS, 클라우드 인프라 확대 및 AI 전문 서비스 제공 강화",
        "글로벌 투자 유치 활성화, 국내 벤처기업 성장 동력"
    ],
    "정치": [
        "대선 이후 최초 총정례회, 국민적 합의와 국제협력 주시",
        "반추리 정책 논란 불거짐, 의원들 간 열띤 토론 및 조율 추진",
        "법률 개혁 촉구 시위 진행, 사회적 불평등 해소를 위한 대안 제시",
        "북한의 최근 군사 행동에 대한 국제 사회 경각심 고조, 긴장 완화 노력 강화",
        "외교 관계 개선을 위한 다자간 회담 계획, 한미일 공동 방위 의지 확인"
    ],
    "사회": [
        "온라인 성추행 사건 증가에 대한 대응책 논의 활발, 법 개정 필요성 제기",
        "고령화 사회 심각성 인식, 노인복지 정책 강화 위한 논의 시작",
        "지역 소통 및 공동체 의식 회복을 위한 시민참여 프로그램 확대 추진",
        "환경오염 문제에 대한 국민적 관심 증가, 지속 가능한 발전 방안 모색",
        "교육열 고조, 청소년들의 학업 스트레스 심화… 정신 건강 지원 필요성 강조"
    ],
    "문화": [
        "한국 영화, 해외 시장 진출 확대… 글로벌 팬덤 확보 노력",
        "국내 음악 예술가, 세계적 뮤직 페스티벌 참여 통해 국제적인 위상 제고",
        "인공지능 활용한 새로운 창작 콘텐츠 개발, 문화 산업 발전 동력으로 부상",
        "전통문화 체험 프로그램 확대, 국내 관광산업 활성화에 기여",
        "독립영화 지원 및 공개 시스템 개선, 다양성 있는 영화 제작 환경 조성"
    ],
    "과학": [
        "한국 연구팀, 암 치료 효과 발휘하는 신약 개발 성공… 임상시험 진행 예정",
        "인공지능 기술 활용한 의료진 보조 시스템 개발 추진, 의료 서비스 질 향상 기대",
        "우주탐사 프로그램 확대, 지구 밖 생명체 존재 여부 확인 위한 연구 강화",
        "에너지 효율 높이는 재생에너지 기술 개발, 환경 문제 해결에 기여",
        "바이오공학 분야 혁신, 질병 치료 및 의약품 개발 동력으로 활용"
    ]
}

# DataFrame으로 변환
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv("five_domain.csv", index=False, encoding='utf-8-sig')

### Few-shot

In [21]:
# 사용할 Few-shot예제를 정의합니다.
# 여기서도 마찬가지로 뉴스 제목 생성에 관해 진행하겠습니다.
examples = [
    {
        "domain": "0",
        "answer": [
            '서울에 다시 오존주의보…도심·서북·동북권 발령종합',
            '게시판 KBS 코로나가 바꾼 일상 대국민 영상 공모',
            '영상 냄새가 나는데…퇴근길 마주친 수상한 차 따라가봤더니',
            '한산한 성산일출봉 주차장',
            '크루즈 관광객용 반나절 부산 해안 트레킹 상품 개발',
            '류동, 소리길C 곡으로 새로운 음악적 도전',
            '섬들의 근대화는 달랐다…일제가 쓰고 버린 태평양 섬 이야기',
            '숲속의 숲 해설센터, 청소년 교육 프로그램 운영',
            '어린이날, 과학관서 다채로운 행사로 가족들 웃음꽃 피우다',
            '오늘도 여전히 정선·의성 9.1도, 서울 8도'
        ]
    },
    {
        "domain": "1",
        "answer": [
            '아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...',
            '추신수 타율 0.265로 시즌 마감…최지만은 19홈런·6...',
            '아시안게임 목소리 높인 박항서 베트남이 일본 못 이길...',
            'MBC 류현진 특집 다큐…MLB 전설에게 듣는 류현진',
            '마틴 우리카드 코치로 V리그 복귀…한국 좋아요',
            '대한항공 우리카드 꺾고 3연승…GS칼텍스 1라운드 전승종합',
            'UEFA 챔스리그 4강 마드리드 더비 성사…레알 vs 아...',
            '프로농구 인삼공사 컬페퍼 햄스트링 부상…교체할 듯',
            '최정의 멀티히트 3출루...탬파베이 4연패 탈출',
            '통산 월드컵 4회 우승 이탈리아 축구 대재앙이 찾아왔다'
        ]
    },
    {
        "domain": "2",
        "answer": [
            "정당 간 연합 논의 본격화, 정치 지형에 미칠 영향은",
            '朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합',
            '황총리 각 부처 비상대비태세 철저히 강구해야',
            '北 무수단 발사 대기권 재진입 기술 확보 여부가 관건',
            '北 서방에 백두산 연구 먼저 제의…과학외교 활발',
            '문대통령 호주 총리와 축구 평가전 1대1 무승부 축하',
            '영상 선출부터 퇴장까지…나경원 원내대표 파란만장 1년',
            '청와대 21일, 대통령 발의 헌법 개정안 초안 공개',
            '새누리 원내대표 선출, 여소야대 속 계파 갈등 심화',
            '당대표 위해 꺼낸 브렉시트 국가연합안이 엄격한 결과 초래'
        ]
    },
    {
        "domain": "3",
        "answer": [
            '게시판 KISA 박민정 책임연구원 APTLD 이사 선출',
            '공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예',
            '한국민족의 뿌리가 북방…5천년 전 한반도서 자생적으로 형성',
            '태블릿PC 조작설 변희재 1심 징역2년…악의적 공격 반복종합',
            '안보리 대북결의안 2270호 이행보고서 제출한 나라 70개 육박',
            '사회정의를 분배 문제에 가두지 말라…지배와 억압 제거해야',
            '교육계 논란, 모멘트 학교 급식 칸막이 설치 문제 재점화',
            '서울, 코로나19 재확산에 따른 방역 대책 강화 필요성 대두',
            '정부, 대부업 대출 규제 강화...금융 소비자 보호 강화',
            '경기권 농산물 생산량 급증, 공급 과잉 우려 커져'
        ]
    },
    {
        "domain": "4",
        "answer": [
            '갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩',
            "유전자 편집 기술의 미래: CRISPR의 혁신과 윤리적 논의",
            '듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감',
            '페이스북 인터넷 드론 아퀼라 실물 첫 시험비행 성공',
            '근육조직 생성에 필수적인 열쇠 발견',
            'LG G8 씽큐 15일부터 예판…디스플레이 1회 무상교체 혜택',
            '네이버 모바일 연예판에도 AI 콘텐츠 추천 시스템 적용',
            '공룡 파충류보다는 타조·매 등 새에 가깝다',
            '삼성전자, AI 기반 새 버전 공개로 업무 효율성 극대화',
            'SK IBM 왓슨, 인공지능 브랜드 에이브AI 출범'
        ]
    },
    {
        "domain": "5",
        "answer": [
            "ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다",
            'NH투자 1월 옵션 만기일 매도 우세',
            '월세 전환에 늘어나는 주거비 부담…작년 역대 최고치',
            '동부지방노동청 임금체불 업체 이율 7% 인하',
            '증시신상품 국채선물 인버스 ETF 3종',
            '흥국운용 2천350억원 중순위 인수금융펀드 1호 결성 완료',
            '웨이포트 6월 22일 상장폐지 신청 예정',
            '초미세먼지로 인한 물가 상승, 한국 경제의 새로운 걸림돌',
            '한투증권 빙그레 폭염에 빙과 수요 증가…매수',
            '11월 주식 발행액 전월비 60% 증가…IPO·유상증자↑'
        ]
    },
    {
        "domain": "6",
        "answer": [
            '美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다',
            '피폭 노동자가 증언하는 후쿠시마 원전 은폐와 속임수',
            '獨 反이스라엘 테러 팔레스타인인 강연 차단…비자도 취소',
            '대한항공 인천∼바르셀로나 노선 첫 취항',
            '유엔 리비아 내전 악화·국제적 확산 우려…아랍연맹 긴급회의종합',
            '러, 미사일 발사 중단 요구에 4국 반응 주목',
            '日 자민당, 헌법 개정 추진...세금 유산 문제 해결 모색',
            '이스라엘 총리, 이란과의 긴장 고조 속 군사 대비 태세 강화',
            '美 항소법원, 인터넷 명령 합헌 판결...사이버 보안 강화',
            '이탈리아, 콘테 총리에게 권력 집중...새 연정 구성 완료'
            
        ]
    }
]

# 입력할 도메인, 그리고 답변을 PromptTemplate에 입력합니다.
example_prompt = PromptTemplate.from_template(
    "Domain:\n{domain}\nAnswer:\n{answer}"
)

# example, example_prompt, input_variables를 활용해주세요
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Domain:\n{domain}\nAnswer:",
    input_variables=["domain"],
)

In [22]:
domain = "0"
final_prompt = prompt.format(domain=domain)
answer = llm.invoke(final_prompt)

In [23]:
answer

"It appears you are trying to provide examples of news articles categorized into different domains. This is a great dataset for training a text classification model! \n\nHere's how we can analyze and utilize this data:\n\n**Understanding the Domains:**\n\n* **0: General News:**  This likely encompasses a broad range of topics, including local events, national news, international affairs, politics, business, and more.\n* **1: Sports:** Articles about various sports, athletes, teams, and sporting events.\n* **2: Politics:** News focused on government, elections, political parties, policy, and current events with a political focus.\n* **3: Technology:**  Articles covering technological advancements, gadgets, software, science, and internet-related news.\n* **4: Business/Finance:** Financial markets, stock prices, companies, economic trends, and business news.\n* **6: International News:** Global events, foreign policy, international relations, and news from around the world.\n\n**Using th

___

In [11]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from typing import List, Tuple

def create_headline_generator():
    """
    뉴스 헤드라인 생성기를 초기화합니다.
    """
    # Ollama Gemma 모델 초기화
    model = Ollama(
        model="gemma2",
        temperature=0.7,
    )
    
    # 프롬프트 템플릿 설정
    prompt = PromptTemplate(
        input_variables=["domain"],
        template="""너는 숙련된 뉴스 편집자야. 주어진 텍스트와 주제 번호를 바탕으로 적절한 뉴스 헤드라인을 작성해야 해.
        
        ### 다음 지침을 따라야해.
        - 제시된 주제 번호에 해당하는 예시 헤드라인들의 내용을 통해 주제를 파악하세요.
        - 파악한 주제에 맞는 새롭고 창의적인 헤드라인을 작성하세요.
        - 생성된 헤드라인은 예시의 내용을 포함하지 않아야 합니다.
        - 설명은 출력하지 마세요.
        
        ### 주제 번호 0 예시:
        예시 1: 도자기 작품 감상하는 손님들
        예시 2: 여행의 매력, 제주에서 다양한 취향을 만족시키다...문화 축제
        예시 3: 공지 1년 만의 소설 출간, 할머니는 죽지 않는다
        예시 4: 한국 코믹콘 축제 개막...다양한 콘텐츠 선보여
        예시 5: 숲속의 숲 해설센터, 청소년 교육 프로그램 운영
        예시 6: 인제 자작나무 숲 입산 통제 15일부터 2개 구간
        예시 7: 6월 5일 날씨: 낮 최고 8도, 강릉 미세먼지 나쁨
        예시 8: 세종시 금정문화권 2단계 개발 사업 7월 착수 문화제 개최
        예시 9: 오늘도 여전히 정선·의성 9.1도, 서울 8도
        예시 10: 스타필드 하남 개장 후 이틀 동안 3만 명 방문
        
        ### 주제 번호 1 예시:
        예시 1: 프로농구 선수 윤호영, 무릎 수술 비용 650만원 지원
        예시 2: GD 감독, 2로축구연맹 월드컵 7개국 순회 경기 개최
        예시 3: 메시, 바르셀로나 떠나 빌바오 입단
        예시 4: MLB 로키스, 피츠버그와 경기! 뉴에이지 메달리스트 감독 취임
        예시 5: 삼성이 극적인 역전을... 손혁 감독, 연패 늪에서 벗어나
        예시 6: 올림픽 상비군 대표팀, 베트남과 경기 8일 오후 7시 개최
        예시 7: 아약스 맨시티 677골...28년 구단 최다수 득점자
        
        ### 주제 번호 2 예시:
        예시 1: 박원주 공정거래위원회 위원장, 누리과정 문제 해결 위해 소통하자는데 반응 주목
        예시 2: 문재인 정부, 민간 2동 7사위 철거 명령
        예시 3: 선거 여론조사 결과 유출 논란
        예시 4: 친박 원내대표 표오도원, 유승민과 결별...단일화 실패(종합)
        예시 5: 남북정상회담 준비가 생명, 남북 리허설 또 리허설
        예시 6: 총선 5개월 앞인데 선거운동 4정 안갯속...정치인들 발벗고 나서
        예시 7: 트럼프/ 일본 방문; 북한과 무역적자 문제 논의 32
        예시 8: 김부겸 전 총리 이달 안 결정...단일 후보 앞으로 나아가야
        예시 9: 민주화운동기념법안 9일 난항...고민 중인 여당, 야당은 반대
        예시 10: 시진핑 중국 대표와 인도 총리 회담
        
        ### 주제 번호 3 예시:
        예시 1: 모멘트 학교 급식 칸막이 설치 논란
        예시 2: 언론인, 조부모, 장애인...시각장애인 폭 넓게 포용하는 시인
        예시 3: 강원 전교조 교사들 원스트라이크 아웃 제도 도입 반발
        예시 4: 코로나19 환자 36일째 단 한 명도 입원하지 않아 건강 악화, 병원들 비상 (종합)
        예시 5: 학교폭력 가해자 엄벌 요구하는 학생들
        예시 6: 광주 북구, 청소년 계도 프로그램 시작
        예시 7: 정부, '인터넷 중독' 청소년 100만 명 구조 요청
        예시 8: 종교시설 방역수칙 위반 사업자 증가...교황청 지침 존재한
        예시 9: 한국노총 주 4일제 일자리 논의, 주노동자들 의견 수렴 중
        예시 10: 어린이 보호구역 안전 강화...단속카메라 확대 및 펫티켓 교육
        
        ### 주제 번호 4 예시:
        예시 1: 정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)
        예시 2: 스마트폰 '블루라이트' 막는 필름 출시
        예시 3: 사티아 나델라 인터랙티브 신제품 발표 Y투데이 인터뷰해 미래 먹거리 논의
        예시 4: 갤럭시 S9 퀀텀 패키지 출시, 5천대 한정 판매
        예시 5: 구글 AI 진단은 의사가 할 수 없다...X단 보조 역할
        예시 6: 혈관 속 움막 모더니즘 봇 2개...자율주행로보틱스 연구로
        예시 7: 삼성전자, 새 버전 공개...AI로 업무 효율 높여
        
        ### 주제 번호 5 예시:
        예시 1: LGU+ 통신요금 4만원 이상 고객에게 고금리 적금 출시
        예시 2: 삼성전자, 3분기 영업이익 6조원 감소
        예시 3: 최희림 SK텔레콤 헬릭스비전 병의원 서비스보다 연매출 70% 증가(종합)
        예시 4: 증시 신제품 출시, 펀드 수익률 상승
        예시 5: 비트코인 거래, 올해 50% 이상 증가
        예시 6: 개인투자자들이 소액으로 주식에 투자하는 '개미'들이 시장에서 큰 역할을 하고 있다
        예시 7: 지난달 분양 3천268채, 7개월 연속 증가세
        예시 8: 포스텍, ICT 융합 로보틱스 스타트업에 600억 투자
        
        ### 주제 번호 6 예시:
        예시 1: 美 MBA 여성 비율 계속 증가, 올해 입학생 중 40%가 여성
        예시 2: 프랑스 마크롱 대통령 G7 정상회의 참석
        예시 3: 우크라이나, 러시아에 맞서 '전술핵 맞설 무기' 개발 추진
        예시 4: 사우디 왕세자와 미국 방문...유시엔조 지속 합의종합
        예시 5: 軍 "美대선 앞두고 北 무수단 발사 대비해 감시 강화"
        예시 6: 시진핑 북한 방문 민생 행보 IT 업계까지 '만반 준비'
        예시 7: 이스라엘, 팔레스타인 가자지구에 로켓 발사 대응 태세 강화
        예시 8: 미국 제재 유예 불허에 원유 수출 목 조르는 이란, 호르무즈 해협 봉쇄 경고
        예시 9: 전 세계적으로 기후 변화에 대한 우려가 커지고 있어
        예시 10: 이란 우라늄 원심분리기 가동...합의 이행소종합
        
        ### 뉴스 헤드라인을 다음과 같은 형식으로 2개씩 제시해줘.
        1.헤드라인: [주제에 맞춰 작성된 헤드라인]
        2.헤드라인: [주제에 맞춰 작성된 헤드라인]
        
        ### 위의 정보를 바탕으로 새로운 뉴스 헤드라인을 작성해줘.
        주제 번호: {domain}
        """
    )

    return model, prompt

def generate_headlines(model: Ollama, 
                      prompt: PromptTemplate, 
                      domain: int) -> Tuple[str, str]:
    """
    주어진 도메인에 대한 뉴스 헤드라인을 생성합니다.
    
    Args:
        model: 초기화된 Ollama 모델
        prompt: 프롬프트 템플릿
        domain: 뉴스 도메인 번호 (0-6)
    
    Returns:
        생성된 두 개의 헤드라인
    """
    # 입력값 검증
    if not 0 <= domain <= 6:
        raise ValueError("도메인 번호는 0에서 6 사이여야 합니다.")
    
    # 프롬프트 포맷팅
    formatted_prompt = prompt.format(domain=domain)
    
    # 헤드라인 생성
    response = model.invoke(formatted_prompt)
    
    # 응답 파싱
    lines = response.strip().split('\n')
    headline1 = lines[0].replace('1.헤드라인:', '').strip()
    headline2 = lines[1].replace('2.헤드라인:', '').strip()
    
    return headline1, headline2

def main():
    # 생성기 초기화
    model, prompt = create_headline_generator()
    
    # 각 도메인에 대해 헤드라인 생성
    for domain in range(7):
        try:
            headline1, headline2 = generate_headlines(model, prompt, domain)
            print(f"\n도메인 {domain}의 헤드라인:")
            print(f"1. {headline1}")
            print(f"2. {headline2}")
        except Exception as e:
            print(f"도메인 {domain} 처리 중 오류 발생: {str(e)}")

if __name__ == "__main__":
    main()


도메인 0의 헤드라인:
1. ## 주제 번호: 0
2. 

도메인 1의 헤드라인:
1. ### 주제 번호 1
2. 

도메인 2의 헤드라인:
1. 1. **헤드라인:** 블루라이트 차단 필름, 스마트폰 사용시간 줄이고 눈 건강 보호!
2. 2. **헤드라인:**  새로운 '블루라이트' 필름 출시…눈 피로감 해소

도메인 3의 헤드라인:
1. ## 주제 번호 3:
2. 

도메인 4의 헤드라인:
1. 1. 헤드라인: **삼성전자, AI 업무 효율 향상 버전 공개하며 미래 기술 경쟁 선점**
2. 2. 헤드라인: **갤럭시 S9 퀀텀 패키지 출시…5천대 한정 판매로 팬덤 열광 확산**

도메인 5의 헤드라인:
1. 1. 헤드라인: **"개미 투자 열풍, 증시 리더로 부상...소액 투자 시장 활성화"**
2. 2. 헤드라인: **삼성전자, 3분기 영업이익 감소 속에도 '혁신' 주력…AI·5G 기술 확대 투자**

도메인 6의 헤드라인:
1. 1. 헤드라인: **시진핑 북한 방문, IT 업계까지 '만반 준비'… 중국-북한 경협 가속화 시사**
2. 2. 헤드라인: **미국 제재 유예 불허에 이란, 호르무즈 해협 봉쇄 경고...중동 지역 불안감 고조**


___

## 도메인 특정하지 않았을 때

In [14]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from typing import List

def create_news_title_generator():
    """
    뉴스 제목 생성기를 초기화합니다.
    """
    # Ollama 모델 초기화
    model = Ollama(
        model="gemma2",
        temperature=0.7,
    )
    
    # 프롬프트 템플릿 설정
    prompt = PromptTemplate(
        input_variables=[],  # 입력 변수 없음
        template="""당신은 숙련된 뉴스 편집자입니다. 다양한 분야의 현실적이고 구체적인 뉴스 기사 제목 10개를 생성해주세요.

다음 지침을 따라주세요:
1. 각 제목은 실제 뉴스처럼 구체적이고 현실적이어야 합니다
2. 클릭베이트나 과장된 표현을 사용하지 마세요
3. 각 제목은 한 줄에 하나씩, 번호를 붙여서 작성해주세요
4. 다양한 분야(사회, 문화, 과학, 교육 등)의 뉴스를 포함해주세요
5. 제목 앞에는 1. 2. 3. 과 같이 숫자와 점을 붙여주세요

예시 형식:
1. [뉴스 제목]
2. [뉴스 제목]
3. [뉴스 제목]

제목을 생성해주세요:"""
    )

    return model, prompt

def generate_news_titles(count: int = 10) -> List[str]:
    """
    뉴스 제목을 생성하는 함수
    
    Args:
        count (int): 생성할 뉴스 제목의 수 (기본값: 10)
    
    Returns:
        List[str]: 생성된 뉴스 제목 리스트
    """
    try:
        # 생성기 초기화
        model, prompt = create_news_title_generator()
        
        # 프롬프트 포맷팅
        formatted_prompt = prompt.format()
        
        # 제목 생성
        response = model.invoke(formatted_prompt)
        
        # 응답을 줄 단위로 분리하고 빈 줄 제거
        titles = [line.strip() for line in response.split('\n') if line.strip()]
        
        # 지정된 수만큼만 반환
        return titles[:count]
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return []

def main():
    print("뉴스 제목 생성을 시작합니다...\n")
    
    # 뉴스 제목 생성
    titles = generate_news_titles()
    
    # 결과 출력
    if titles:
        print("생성된 뉴스 제목:")
        for title in titles:
            print(title)
    else:
        print("뉴스 제목 생성에 실패했습니다.")

if __name__ == "__main__":
    main()

뉴스 제목 생성을 시작합니다...

생성된 뉴스 제목:
1. **고령 인구 증가에 따라 노인 돌봄 서비스 수요 급증, 정부 지원 강화 논의**
2. **'비디오 게임 개발 전문 교육 과정', 대학에서 시작 앞두러**
3. **기상청, 내일 서울 등 중부 지방 지역 호우 경보 발령 예상**
4. **국내 최초 '암세포 치료용 바이오 칩' 개발 성공, 임상시험 진입 목표**
5. **올해 국내 여행수 전년 대비 20% 증가, 코로나 이후 '여행 열풍' 지속**
6. **‘반짝이는 도시’를 위한 스마트 불빛 시스템 도입, 에너지 절약 효과 기대**
7. **인공지능 기술 활용으로 의료 진단 정확도 향상, 새로운 치료법 개발 가능성 제기**
8. **전국 공공 도서관의 디지털 자료 접근성 확대, 온라인 학습 환경 개선**
9. **'유튜브 인플루언서' 활동으로 얻는 수익, 정부 감시 대상으로 논란 발생**
10. **자연재해 피해 지역에 대한 재건 지원금 조달, 국제 사회의 적극적 참여 독려**


In [15]:
titles = generate_news_titles(count=100)

In [6]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from typing import List
import pandas as pd
import os
from datetime import datetime

def create_news_title_generator():
    """
    뉴스 제목 생성기를 초기화합니다.
    """
    # Ollama 모델 초기화
    model = Ollama(
        model="gemma2",
        temperature=0.7,
    )
    
    # 프롬프트 템플릿 설정
    prompt = PromptTemplate(
        input_variables=[],
        template="""당신은 숙련된 뉴스 편집자입니다. 다양한 분야의 현실적이고 구체적인 뉴스 기사 제목 100개를 생성해주세요.

다음 지침을 따라주세요:
1. 각 제목은 실제 뉴스처럼 구체적이고 현실적이어야 합니다
2. 클릭베이트나 과장된 표현을 사용하지 마세요
3. 각 제목은 한 줄에 하나씩, 번호를 붙여서 작성해주세요
4. 다양한 분야(사회, 문화, 과학, 교육 등)의 뉴스를 포함해주세요
5. 제목 앞에는 1. 2. 3. 과 같이 숫자와 점을 붙여주세요

예시 형식:
1. [뉴스 제목]
2. [뉴스 제목]
3. [뉴스 제목]

제목을 생성해주세요:"""
    )

    return model, prompt

def generate_news_titles(count: int = 100) -> List[str]:
    """
    뉴스 제목을 생성하는 함수
    
    Args:
        count (int): 생성할 뉴스 제목의 수 (기본값: 100)
    
    Returns:
        List[str]: 생성된 뉴스 제목 리스트
    """
    try:
        model, prompt = create_news_title_generator()
        formatted_prompt = prompt.format()
        response = model.invoke(formatted_prompt)
        
        # 응답을 줄 단위로 분리하고 번호 제거
        titles = []
        for line in response.split('\n'):
            line = line.strip()
            if line:
                # 번호와 점(.) 제거 후 앞뒤 공백 제거
                title = '.'.join(line.split('.')[1:]).strip()
                if title:  # 빈 문자열이 아닌 경우만 추가
                    titles.append(title)
        
        return titles[:count]
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return []

def save_titles_to_csv(titles: List[str], filename: str = None) -> str:
    """
    생성된 뉴스 제목들을 CSV 파일로 저장합니다.
    
    Args:
        titles (List[str]): 저장할 뉴스 제목 리스트
        filename (str, optional): 저장할 파일명. None이면 자동 생성
    
    Returns:
        str: 저장된 파일의 경로
    """
    try:
        # 제목들을 DataFrame으로 변환
        df = pd.DataFrame({'text': titles})
        
        # 파일명이 지정되지 않은 경우 자동 생성
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"news_titles_{timestamp}.csv"
        
        # CSV 파일로 저장
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        
        return filename
    
    except Exception as e:
        print(f"CSV 저장 중 오류 발생: {str(e)}")
        return None

def main():
    print("뉴스 제목 생성을 시작합니다...\n")
    
    # 뉴스 제목 생성
    titles = generate_news_titles()
    
    if titles:
        print("생성된 뉴스 제목:")
        for i, title in enumerate(titles, 1):
            print(f"{i}. {title}")
        
        # CSV 파일로 저장
        saved_file = save_titles_to_csv(titles)
        if saved_file:
            print(f"\n제목들이 성공적으로 저장되었습니다: {saved_file}")
        else:
            print("\nCSV 파일 저장에 실패했습니다.")
    else:
        print("뉴스 제목 생성에 실패했습니다.")

In [ ]:
titles = generate_news_titles()

'my_news_titles_1.csv'

In [ ]:
news_100 = pd.read_csv('my_news_titles.csv')
news_all = pd.read_csv('my_news_titles_all.csv')
news = pd.concat([news_100, news_all], ignore_index=True)
save_titles_to_csv(all_titles, "news_titles_notdomain.csv")

'news_titles_notdomain.csv'